In [14]:
#Import the necessary libraries
#To read files locally
import json
import os
#To manipulate data + PyTerrier
import re
import calendar
import locale
import pandas as pd
import pyterrier as pt

In [15]:
if not pt.started():
  pt.init()

In [16]:
# Path to the JSON file
data_file_path = "../indexing/db/make_db2.json"

# Load the data from the JSON file
with open(data_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [17]:
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

for entry in data:
    entry["text"] = entry["text"].strip('“”')  # Eliminar comillas al inicio y al final
    
    if entry["tags"]:
        # print("ENTRY TAG IS", type(entry["tags"]))
        concat = ""
        for elem in entry["tags"]:
            concat += elem + " "
        entry["tags"] = concat

    if 1 <= entry["month"] <= 12:
        month_name = calendar.month_name[entry["month"]]
        entry["month"] = month_name + " " + str(entry["month"])

for entry in data:
    # print("ENTRY", entry)
    text = "{} {} {} {} {} {}".format(
        entry["text"], 
        entry["title"],
        entry["author"], 
        entry["tags"],
        entry["year"],
        entry["month"])
    # print("text 1", text)
    text = re.sub(r'[^a-zA-Z0-9]', ' ',text.replace("None", " "))
    # print("text 2", text)
    text = re.sub(r'\s{2,}', ' ', text)
    entry["text"] = text
    # print("etnry last ", entry["text"])

In [18]:

docs_df = pd.DataFrame(pd.DataFrame([{'text': entry['text']} for entry in data]))

docnumber = ["d" + str(x) for x in range(1,len(docs_df) + 1)]

docs_df.insert(0, 'docno', docnumber)

docs_df = docs_df.dropna(subset=['text'])

dict_array = []

for index, rows in docs_df.iterrows():
    row_dict = {}
    row = docs_df.iloc[index]
    row_dict["docno"] = row["docno"]
    row_dict["text"] = row["text"]
    dict_array.append(row_dict)

with open('./db/output.json', 'w') as json_file:
    json.dump(dict_array, json_file, indent=2)

# # Assuming docs_df is your DataFrame
# docs_df.to_json('./db/docs_df.json', orient='records', lines=True, force_ascii=True)

# docs_df.style
docs_df


,docno,text
0,d1,In our interview with Janne Aas Jakobsen Found...
1,d2,In an era where the digital landscape is const...
2,d3,In this insightful interview we explore the en...
3,d4,Making wise decisions and staying ahead of the...
4,d5,In the quickly developing field of artificial ...
...,...,...
7555,d7556,Welcome to Analytics Vidhya Welcome to Analyti...
7556,d7557,Appropriate space allocation and compelling vi...
7557,d7558,Business Analytics has become a catch all word...
7558,d7559,Joining Merging in SAS alternate approaches i...


In [19]:
# IF THIS ISN'T RUNNING
# TRY THIS
# !pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

preIndexTable = pd.read_json('./db/output.json')
indexer = pt.DFIndexer('./index_ai', overwrite=True)
index_ref = indexer.index(preIndexTable["text"], preIndexTable["docno"])
index_ref.toString()
index = pt.IndexFactory.of(index_ref)

In [20]:
print(index.getCollectionStatistics().toString())

Number of documents: 7560
Number of terms: 12523
Number of postings: 218416
Number of fields: 0
Number of tokens: 286459
Field names: []
Positions:   false



In [21]:
pd.DataFrame([(kv.getKey(), kv.getValue().toString()) for kv in index.getLexicon()]).tail()


,0,1
12518,zquez,term5609 Nt=1 TF=1 maxTF=1 @{0 259788 1}
12519,zscaler,term8575 Nt=1 TF=1 maxTF=1 @{0 259790 7}
12520,zuckerberg,term12101 Nt=1 TF=1 maxTF=1 @{0 259793 7}
12521,zue,term3881 Nt=1 TF=1 maxTF=1 @{0 259797 1}
12522,zurich,term5610 Nt=1 TF=1 maxTF=1 @{0 259799 5}


In [22]:
for kv in index.getLexicon():
  print("%s  -> %s " % (kv.getKey(), kv.getValue().toString()  ))

0  -> term1274 Nt=65 TF=124 maxTF=5 @{0 0 0} 
00  -> term3256 Nt=5 TF=6 maxTF=2 @{0 100 5} 
000  -> term1834 Nt=36 TF=38 maxTF=2 @{0 112 4} 
004second  -> term10809 Nt=1 TF=1 maxTF=1 @{0 172 4} 
007  -> term6155 Nt=16 TF=26 maxTF=3 @{0 175 6} 
01  -> term10587 Nt=4 TF=4 maxTF=1 @{0 192 0} 
02  -> term10366 Nt=1 TF=1 maxTF=1 @{0 202 0} 
03  -> term11083 Nt=1 TF=1 maxTF=1 @{0 205 2} 
036  -> term4225 Nt=1 TF=1 maxTF=1 @{0 208 4} 
05  -> term4546 Nt=2 TF=3 maxTF=2 @{0 211 0} 
055  -> term2023 Nt=1 TF=1 maxTF=1 @{0 215 7} 
06  -> term5082 Nt=1 TF=1 maxTF=1 @{0 218 1} 
07  -> term5081 Nt=2 TF=2 maxTF=1 @{0 220 5} 
08  -> term10365 Nt=1 TF=1 maxTF=1 @{0 225 3} 
092  -> term7875 Nt=1 TF=1 maxTF=1 @{0 228 5} 
1  -> term627 Nt=557 TF=583 maxTF=4 @{0 231 5} 
10  -> term96 Nt=826 TF=858 maxTF=3 @{0 548 7} 
100  -> term1692 Nt=32 TF=37 maxTF=3 @{0 988 7} 
1000  -> term11731 Nt=1 TF=1 maxTF=1 @{0 1046 6} 
1000th  -> term11410 Nt=1 TF=1 maxTF=1 @{0 1050 0} 
1000x  -> term10911 Nt=2 TF=3 maxTF=2 @{0 

In [23]:
br = pt.BatchRetrieve(index, wmodel="BM25") #Alternative Models: "TF_IDF", "BM25"
br.search("mark zuckerberg")

,qid,docid,docno,rank,score,query
0,1,7031,d7032,0,22.840681,mark zuckerberg
1,1,1624,d1625,1,10.407073,mark zuckerberg
2,1,459,d460,2,9.125934,mark zuckerberg
3,1,1143,d1144,3,8.828652,mark zuckerberg
4,1,5884,d5885,4,8.689975,mark zuckerberg
5,1,121,d122,5,8.487250,mark zuckerberg
6,1,277,d278,6,8.389394,mark zuckerberg
7,1,337,d338,7,8.389394,mark zuckerberg
8,1,7179,d7180,8,8.293769,mark zuckerberg
9,1,6178,d6179,9,8.108913,mark zuckerberg


In [24]:
def retrieve_info(cdf, db_objects):
    article_title = []
    article_author = []
    # article_year = []
    for i in range(cdf.shape[0]):
        docId = cdf.loc[i, "docno"]
        docNo = int(docId[1:]) - 1
        article_title.append(db_objects[docNo]["title"])
        article_author.append(db_objects[docNo]["author"])
        # article_year.append(db_objects)[docNo]["year"]
    cdf["title"] = article_title
    cdf["author"] = article_author
    # cdf["year"] = article_year
    
    return cdf


In [25]:
def getQueryResult(index, query, db_objs):
    bm25 = pt.BatchRetrieve(index, num_results=10, wmodel="BM25")

    queries = pd.DataFrame(
        query,
        columns=["qid", "query"],
    )

    results = bm25.transform(queries)
    formatedResult = retrieve_info(results, db_objs)
    print(formatedResult)

In [26]:
db_objs = []

with open('../indexing/db/make_db2.json', "r") as f:
    objects = json.load(f)
    for obj in objects:
        db_objs.append(obj)


preIndexTable = pd.read_json('../indexing/db/output.json')
indexer = pt.DFIndexer('./index_ai', overwrite=True)
index_ref = indexer.index(preIndexTable["text"], preIndexTable["docno"])
index_ref.toString()


query = [["q1", "data analysis"], ["q2", "gemini"]]
getQueryResult(index, query, db_objs)


query1 = [["q1", "bard"], ["q2", "openai"]]
getQueryResult(index, query1, db_objs)

   qid  docid  docno  rank      score          query  \
0   q1   5727  d5728     0   6.337105  data analysis   
1   q1   3453  d3454     1   6.325797  data analysis   
2   q1   4574  d4575     2   6.184188  data analysis   
3   q1   4086  d4087     3   6.064109  data analysis   
4   q1   6925  d6926     4   6.052351  data analysis   
5   q1   7333  d7334     5   6.024530  data analysis   
6   q1   6922  d6923     6   5.978513  data analysis   
7   q1   7444  d7445     7   5.966052  data analysis   
8   q1   7478  d7479     8   5.942265  data analysis   
9   q1   3044  d3045     9   5.925027  data analysis   
10  q2   1646  d1647     0  16.544766         gemini   
11  q2   1668  d1669     1  14.504162         gemini   
12  q2   2274  d2275     2  13.157922         gemini   
13  q2   1776  d1777     3  10.258529         gemini   

                                                title             author  
0   Exploratory Analysis Using Univariate, Bivaria...        Khushi Shah  
1      Se